<a href="https://colab.research.google.com/github/karencfisher/hotel-reviews/blob/main/notebooks/hotel_classification_quick_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
!wget https://raw.githubusercontent.com/karencfisher/hotel-reviews/main/data/reviews_sample.csv

--2023-02-13 19:28:44--  https://raw.githubusercontent.com/karencfisher/hotel-reviews/main/data/reviews_sample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23866047 (23M) [text/plain]
Saving to: ‘reviews_sample.csv’

reviews_sample.csv  100%[===================>]  22.76M  --.-KB/s    in 0.1s    

2023-02-13 19:28:44 (155 MB/s) - ‘reviews_sample.csv’ saved [23866047/23866047]



In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import hamming_loss, accuracy_score 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df_reviews = pd.read_csv('..\\data\\reviews_sample.csv')
df_reviews.drop(columns=['Overall', 'Location'], inplace=True)
df_reviews

,Cleanliness,Rooms,Service,Sleep Quality,Value,Content,average_score,sentiment
0,4,3,4,2,5,excellent value drawbacks second trip rius pla...,3.428571,0
1,2,3,2,3,3,great location hotel could better okay planned...,3.000000,0
2,3,2,2,2,3,slightly better dingy booked via hotelscom rea...,2.857143,0
3,5,2,2,5,2,could worse also much better bad hotel ever st...,3.142857,0
4,3,1,3,3,2,faded glory nice try place embarrassment marri...,2.571429,0
...,...,...,...,...,...,...,...,...
35117,5,3,1,4,5,say liked price cleaning service didnt pay muc...,3.571429,1
35118,4,4,4,4,5,affordable cleanm convenient touch atmosphere ...,4.142857,1
35119,5,5,5,5,5,great stay would gladly go back free copies ny...,5.000000,1
35120,5,5,5,5,5,resort hotel great value terrific place conven...,5.000000,1


Convert ratings to tags for Cleanliness, Rooms, Service, and Value. 1 if less than rating of 4 to indicate in need of attention.

In [3]:
tags = ['Cleanliness', 'Rooms', 'Service', 'Sleep Quality', 'Value']
df_reviews[tags].astype(int)

for tag in tags:
  df_reviews[tag] = df_reviews[tag].apply(lambda x: 1 if x < 4 else 0)

df_reviews

,Cleanliness,Rooms,Service,Sleep Quality,Value,Content,average_score,sentiment
0,0,1,0,1,0,excellent value drawbacks second trip rius pla...,3.428571,0
1,1,1,1,1,1,great location hotel could better okay planned...,3.000000,0
2,1,1,1,1,1,slightly better dingy booked via hotelscom rea...,2.857143,0
3,0,1,1,0,1,could worse also much better bad hotel ever st...,3.142857,0
4,1,1,1,1,1,faded glory nice try place embarrassment marri...,2.571429,0
...,...,...,...,...,...,...,...,...
35117,0,1,1,0,0,say liked price cleaning service didnt pay muc...,3.571429,1
35118,0,0,0,0,0,affordable cleanm convenient touch atmosphere ...,4.142857,1
35119,0,0,0,0,0,great stay would gladly go back free copies ny...,5.000000,1
35120,0,0,0,0,0,resort hotel great value terrific place conven...,5.000000,1


Test with a small sample of the data set

In [4]:
y = np.asarray(df_reviews[tags])
x_train, x_test, y_train, y_test = train_test_split(df_reviews['Content'], 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
x_train.shape, y_train.shape

((24585,), (24585, 5))

Construct a consistent pipeline. 

In [57]:
class Model:
  def __init__(self, input_shape):
    self.model = Sequential([Dense(5, 
                             activation='sigmoid', 
                             input_shape=input_shape)])

  def fit(self, x, y, epochs=100):
    self.model.compile(optimizer='adam', loss='mse')
    history=self.model.fit(x, y, epochs=epochs)
    return history

  def predict(self, x):
    return self.model.predict(x)


class CLF:
  def __init__(self, max_features):
    self.vectorize = TfidfVectorizer(max_features=max_features)
    self.clf = Model((max_features,))

  def fit(self, x, y, epochs=100):
    self.vectorize.fit(x)
    x = self.vectorize.transform(x).toarray()
    self.clf.fit(x, y, epochs)
    return self, x

  def predict(self, x, threshold=0.5):
    x = self.vectorize.transform(x).toarray()
    logits =  self.clf.predict(x)
    y = (logits > threshold).astype(int)
    return y, logits


In [58]:
pipe = CLF(500, 5)
_, x = pipe.fit(x_train, y_train, epochs=500)

Epoch 1/500
769/769 [==============================] - 1s 750us/step - loss: 0.2170
Epoch 2/500
769/769 [==============================] - 1s 670us/step - loss: 0.1794
Epoch 3/500
769/769 [==============================] - 1s 674us/step - loss: 0.1616
Epoch 4/500
769/769 [==============================] - 1s 658us/step - loss: 0.1517
Epoch 5/500
769/769 [==============================] - 1s 781us/step - loss: 0.1454
Epoch 6/500
769/769 [==============================] - 1s 688us/step - loss: 0.1410
Epoch 7/500
769/769 [==============================] - 1s 770us/step - loss: 0.1379
Epoch 8/500
769/769 [==============================] - 1s 683us/step - loss: 0.1355
Epoch 9/500
769/769 [==============================] - 1s 739us/step - loss: 0.1336
Epoch 10/500
769/769 [==============================] - 1s 740us/step - loss: 0.1322
Epoch 11/500
769/769 [==============================] - 1s 785us/step - loss: 0.1310
Epoch 12/500
769/769 [==============================] - 1s 722us/step - lo

In [59]:
y_hat, logits = pipe.predict(x_test)
print(logits, y_hat)

[[0.782639   0.956944   0.8711284  0.95936996 0.95169413]
 [0.39869407 0.83083653 0.7712211  0.6057395  0.6923034 ]
 [0.6263255  0.65268785 0.9687918  0.5241167  0.89926434]
 ...
 [0.96023613 0.9751598  0.9734744  0.9646449  0.9757739 ]
 [0.858441   0.9967834  0.90104485 0.8862379  0.965725  ]
 [0.04928195 0.06418735 0.05586532 0.0629009  0.15009195]] [[1 1 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]
 ...
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]]


In [60]:
y_test[:5]

array([[0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int64)

In [61]:
x_test[:5]

4520     one worst rooms ive stayed booked hotel basis ...
13214    mixed feelings im frequent traveler business l...
5706     would great stay except wrong charges bad serv...
29850    good option staff friendly otherwise good leve...
34791    good value clean comfortable hotel located rig...
Name: Content, dtype: object

In [62]:
print(accuracy_score(y_test, y_hat)) 
print(hamming_loss(y_test, y_hat))

0.5171301129353706
0.18555566100408086


In [63]:
myreview = "This is the niceset hotel I have stayed in. The staff was really helpful and kind. The room was spotless. I will recommend this to all my friends!"
result = pipe.predict([myreview])
result

(array([[0, 0, 0, 0, 0]]),
 array([[0.29702133, 0.2500888 , 0.03184918, 0.20705447, 0.14709735]],
       dtype=float32))

In [64]:
badreview = "I had no sleep at all. The room was filthy. The service really did not care at all."
result = pipe.predict([badreview])
result

(array([[1, 1, 1, 1, 1]]),
 array([[0.6717682 , 0.7540729 , 0.7506741 , 0.9095476 , 0.91178936]],
       dtype=float32))

In [65]:
tags

['Cleanliness', 'Rooms', 'Service', 'Sleep Quality', 'Value']

In [66]:
review = "A very nice place except the service is poor."
result = pipe.predict([badreview])
result

(array([[1, 1, 1, 1, 1]]),
 array([[0.6717682 , 0.7540729 , 0.7506741 , 0.9095476 , 0.91178936]],
       dtype=float32))